### Imports and Utils

In [1]:
#change cupy to numpy if cupy not installed
import numpy as np
from scipy.optimize import minimize
from itertools import product
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [ ]:
# credit arxiv 1910.08187
sk_angles = pickle.load(open("data/sk_angles.pickle", "rb"))

In [ ]:
def stretch_array(arr, new_length):
    old_length = len(arr)
    if old_length == new_length:
        return arr  # No change needed
    
    # Create an array of new indices
    new_indices = np.linspace(0, old_length - 1, new_length)
    
    # Interpolate values
    stretched_array = np.interp(new_indices, np.arange(old_length), arr)
    
    return stretched_array


def gen_all_bitstrings(bs_size):
    arrays = [[-1, 1]] * bs_size
    mesh = np.meshgrid(*arrays)
    return np.stack(mesh, axis=-1).reshape(-1, bs_size)

### Compact Iterations at $D\rightarrow \infty$

#### Implementing Ansatze

##### $MC$ Ansatz

In [ ]:
def get_z_idx(p, idx): # assuming a is indexed as 1,2,3...p,0,-p,...-1, find index of idx in a
    return (idx>0)*(idx-1) + (idx==0)*p + (idx<0)*(2*p+idx+1) 

def get_mc_eib_entries(bit1, bit2, beta):    
    return np.cos(beta)**(np.abs(bit1+bit2)/2) * (1j * np.sin(beta))**(np.abs(bit1-bit2)/2)

def get_mc_f_beta(p, bs, beta, get_f_prime=False):
    bs_size = 2*p+1
    all_beta = np.concatenate((beta, np.array([-beta_j for beta_j in beta[::-1]])))
    prod = 0.5 * np.ones(len(bs), dtype='complex')
    for j in range(bs_size-1):
        if j==p-1 and get_f_prime:
            prod *= get_mc_eib_entries(bs[:,j], -bs[:,j+1], all_beta[j])
        else:
            prod *= get_mc_eib_entries(bs[:,j], bs[:,j+1], all_beta[j]) 
    return prod

def get_B0(p):
    half = np.array(list(product(np.array([1,-1]), repeat=p+1)))
    return np.hstack((half, np.flip(half,axis=1)[:,1:]))

def get_ta(p, bs):
    ta = 0
    for r in range(1,p+1):
        r_idx, nr_idx = get_z_idx(p, r), get_z_idx(p, -r)
        if bs[r_idx] != bs[nr_idx]:
            ta = r
    return ta

def get_all_ta_leq_m(p, m):
    left = np.array(list(product(np.array([1,-1]), repeat=m)))
    right = np.array(list(product(np.array([1,-1]), repeat=m)))
    middle_half = np.array(list(product(np.array([1,-1]), repeat=p-m+1)))
    middle = np.hstack((middle_half, np.flip(middle_half,axis=1)[:,1:]))
    # Create meshgrids for indices
    left_idx, middle_idx, right_idx = np.meshgrid(np.arange(left.shape[0]),
                                                np.arange(middle.shape[0]),
                                                np.arange(right.shape[0]), indexing='ij')
    # Combine indices to get all possible combinations
    result = np.column_stack((left[left_idx.ravel()], middle[middle_idx.ravel()], right[right_idx.ravel()]))

    return result

def get_Hm_a(p, a, ta, gamma, G_prev):
    exp_sum_1, exp_sum_2 = 0, 0 
    for sp in range(1, ta+1):
        sp_idx, nsp_idx = get_z_idx(p, sp), get_z_idx(p, -sp)
        exp_sum_1 += gamma[sp_idx]**2 * (1-a[sp_idx]*a[nsp_idx])
        for rp in range(1, sp):
            rp_idx, nrp_idx = get_z_idx(p, rp), get_z_idx(p, -rp)
            exp_sum_2 += gamma[sp_idx] * gamma[rp_idx] * (G_prev[rp_idx, sp_idx]*a[rp_idx]-np.conjugate(G_prev[rp_idx, sp_idx])*a[nrp_idx]) * (a[sp_idx]-a[nsp_idx])
    return np.exp(-exp_sum_1 - exp_sum_2)


def get_Hm_a_vectorized(p, as_array, tas, gamma, G_prev):
    exp_sum_1 = np.zeros(as_array.shape[0], dtype='complex')  # Use complex zeros
    exp_sum_2 = np.zeros(as_array.shape[0], dtype='complex')  # Use complex zeros

    for sp in range(1, np.max(tas) + 1):  # Loop over sp values
        sp_idx = get_z_idx(p, sp)  # Get the index for positive sp
        nsp_idx = get_z_idx(p, -sp)  # Get the index for negative sp

        # Determine which indices to consider based on tas
        valid_indices = tas >= sp

        # Compute contributions to exp_sum_1 and exp_sum_2
        exp_sum_1[valid_indices] += gamma[sp_idx]**2 * (1 - as_array[valid_indices, sp_idx] * as_array[valid_indices, nsp_idx])

        for rp in range(1, sp):  # Loop over rp values
            rp_idx = get_z_idx(p, rp)  # Get the index for positive rp
            nrp_idx = get_z_idx(p, -rp)  # Get the index for negative rp
            
            exp_sum_2[valid_indices] += (gamma[sp_idx] * gamma[rp_idx] * 
                (G_prev[rp_idx, sp_idx] * as_array[valid_indices, rp_idx] -
                np.conjugate(G_prev[rp_idx, sp_idx]) * as_array[valid_indices, nrp_idx]) *
                (as_array[valid_indices, sp_idx] - as_array[valid_indices, nsp_idx]))

    return np.exp(-exp_sum_1 - exp_sum_2)  # Return the result as an array



def get_mc_nu(p, gamma, beta, bitstrings, ta, separate=False):

    bs_size=2*p+1
    # Initialize matrices
    G_prev = np.eye(bs_size, dtype='complex') + np.flip(np.eye(bs_size, dtype='complex'), axis=1)
    G_curr = np.eye(bs_size, dtype='complex') + np.flip(np.eye(bs_size, dtype='complex'), axis=1)
    G_prev[p, p], G_curr[p, p] = 1, 1
    G_prime_0 = np.zeros(p, dtype='complex')
    # Precompute fas and fpas
    fas = get_mc_f_beta(p, bitstrings, beta)
    fpas = get_mc_f_beta(p, bitstrings, beta, get_f_prime=True)
    for m in range(1, p + 1):
        m_idx = get_z_idx(p, m)
        valid_indices = np.where(ta <= m)[0]  

        if valid_indices.size > 0:
            valid_bitstrings = bitstrings[valid_indices]
            fa_valid = fas[valid_indices]
            fpa_valid = fpas[valid_indices]
            ta_valid = ta[valid_indices]
            # Compute Hma for all valid bitstrings in a vectorized manner
            # Use a vectorized implementation of get_Hm_a if possible.
            #Hma = np.array([get_Hm_a(p, a, int(ta_i), gamma, G_prev) for a, ta_i in zip(valid_bitstrings, ta_valid)])
            Hma = get_Hm_a_vectorized(p, valid_bitstrings, ta_valid, gamma, G_prev)
            # Use broadcasting to compute contributions in a vectorized manner
            for r in range(1, m + 1):
                r_idx = get_z_idx(p, r)
                if m <= p - 1:
                    # This computation may be done more efficiently with broadcasting
                    G_curr[r_idx, m_idx + 1] += np.sum(fa_valid * Hma * valid_bitstrings[:, r_idx] * valid_bitstrings[:, m_idx + 1])
                else:
                    zero_idx = get_z_idx(p, 0)
                    G_curr[zero_idx, r_idx] += np.sum(fa_valid * Hma * valid_bitstrings[:, zero_idx] * valid_bitstrings[:, r_idx])
                    G_prime_0[r_idx] += np.sum(fpa_valid * Hma * valid_bitstrings[:, zero_idx] * valid_bitstrings[:, r_idx])

        if m <= p - 1:
            G_prev = G_curr
    nu_y, nu_z = 0, 0 
    for r in range(1,p+1):
        r_idx, zero_idx = get_z_idx(p, r), get_z_idx(p, 0)
        nu_y += (-1j/2) * gamma[r_idx] * ((G_prime_0[r_idx])**2 - np.conjugate(G_prime_0[r_idx])**2)
        nu_z += (1j/2) * gamma[r_idx] * ((G_curr[zero_idx, r_idx])**2 - np.conjugate(G_curr[zero_idx, r_idx])**2)		

    if separate: 
        return np.real(nu_y), np.real(nu_z)
    else:
        return np.real(nu_y + nu_z)
    return


def get_mc_nu_helper(p, separate=False, negate=False): 
    # in order to optimize over params
    bs_size = 2*p+1
    bitstrings = gen_all_bitstrings(bs_size)
    # Pre-compute ta for all bitstrings
    ta = np.array([get_ta(p, a) for a in bitstrings])
    def get_expectation(params):
        gamma = params[:p]
        beta = params[p:]
        return (1-2*negate)*get_mc_nu(p, gamma, beta, bitstrings, ta, separate=separate)

    return get_expectation

def get_mc_nu_last_gamma_helper(p, gamma_pm1, beta_pm1, separate=False, negate=False): 
    # in order to optimize over params assuming all but last gamma is fixed
    # input length p-1 gamma and length p-1 beta (last beta will be set to 0)
    bs_size = 2*p+1
    bitstrings = gen_all_bitstrings(bs_size)
    # Pre-compute ta for all bitstrings
    ta = np.array([get_ta(p, a) for a in bitstrings])

    def get_expectation(last_gamma):
        gamma = np.concatenate((gamma_pm1, last_gamma))
        beta = np.concatenate((beta_pm1, np.zeros(1)))
        return (1-2*negate)*get_mc_nu(p, gamma, beta, bitstrings, ta, separate=separate)

    return get_expectation

##### $XY$ ansatz

In [ ]:
# helpers
def get_xy_h_i(y, z):
    # inner product <y|z>
    return 1j**((-y+y*z)/2) / np.sqrt(2)

def get_xy_g_i(y, z, beta):
    # inner product <y|e^{i beta X}|z>
    return get_xy_h_i(y,z) * (np.cos(beta) + (-1)**((y*z-1)/2)*np.sin(beta))

def get_xy_f(p, bitstrings, beta, f_prime=False): # length 2^(4p+1) vector storing result for all a
    # bitstrings,[j,:] indexed as a_1, a_2, ..., a_p, a_0, a_{-p}, .... a_{-1}
    tot = 1/2 * np.ones(2**(4*p+1)).astype('complex')
    for l in range(p):
        tot *= np.conjugate(get_xy_h_i(bitstrings[:,2*l+1], bitstrings[:,2*l])) #<z_l|y_l>
        if f_prime and l==(p-1):
            tot *= get_xy_g_i(bitstrings[:,2*l+1], -bitstrings[:,2*l+2], beta[l]) #<y_l|e^{i beta_l X}|z_{l+1}> (or z_0 if l=p)
        else:
            tot *= get_xy_g_i(bitstrings[:,2*l+1], bitstrings[:,2*l+2], beta[l]) #<y_l|e^{i beta_l X}|z_{l+1}> (or z_0 if l=p)
        tot *= get_xy_h_i(bitstrings[:,-(2*l+2)], bitstrings[:,-(2*l+1)])  #<y_{-l}|z_{-l}>
        tot *= np.conjugate(get_xy_g_i(bitstrings[:,-(2*l+2)], bitstrings[:,-(2*l+3)], beta[l])) #<z_{-(l+1)}|e^{-i beta_l X}|y_{-l}>
    return tot


def get_xy_G2pm1(p, bitstrings, f_a, Gamma_vec, beta):
    # gets G^(2p-1)

    # Initialize G^(0)
    new_G = np.zeros((4 * p + 1, 4 * p + 1), dtype='complex')
    
    # Precompute Gamma_vec outer product
    Gamma_outer = np.outer(Gamma_vec, Gamma_vec)

    for m in range(2 * p):
        prev_G = new_G.copy()
        new_G.fill(0)  # Reset new_G to zero
        # Compute a_outer products for all bitstrings
        a_outer = bitstrings[:, :, np.newaxis] * bitstrings[:, np.newaxis, :]
        # Efficient computation of exp_sums
        exp_sums = np.einsum('ijk,jk->i', a_outer, Gamma_outer * prev_G)
        # Compute exp_factors
        exp_factors = np.exp(-0.5 * exp_sums)
        # Vectorized update of new_G
        new_G += np.tensordot(f_a * exp_factors, a_outer, axes=(0, 0))

    return new_G


def get_xy_H2p(p, bitstrings, Gamma_vec, G2pm1):

    # Precompute Gamma outer product
    Gamma_outer = np.outer(Gamma_vec, Gamma_vec)
    # Compute the term (Gamma_outer * G2pm1) which is constant
    Gamma_G = Gamma_outer * G2pm1
    # Compute the outer product of bitstrings
    a_outer = bitstrings[:, :, np.newaxis] * bitstrings[:, np.newaxis, :]
    # Compute exp_sums using broadcasting and summing over the appropriate axes
    exp_sums = np.sum(a_outer * Gamma_G, axis=(1, 2))
    # Compute H2p
    H2p = np.exp(-0.5 * exp_sums)
    return H2p

def get_xy_nu(p, gamma_y, gamma_z, beta, bitstrings, separate=False):

    Gamma_vec = np.zeros(4*p+1)
    Gamma_vec[:2*p:2]=gamma_z
    Gamma_vec[1:2*p:2]=gamma_y
    Gamma_vec[4*p-1:2*p:-2]=-gamma_y
    Gamma_vec[4*p:2*p:-2]=-gamma_z

    f_a = get_xy_f(p, bitstrings, beta)
    fp_a = get_xy_f(p, bitstrings, beta, f_prime=True)
    G2pm1 = get_xy_G2pm1(p, bitstrings, f_a, Gamma_vec, beta) # get G^(2p-1)
    H2p = get_xy_H2p(p, bitstrings, Gamma_vec, G2pm1) # get H^(2p)

    f_H2p = f_a * H2p
    fp_H2p = fp_a * H2p

    # Vectorize the computation of y_tot and z_tot
    # Shape: (len(bitstrings), 4*p+1)
    y_tot_mat = (fp_H2p[:, np.newaxis] * bitstrings[:, [2*p]] * bitstrings)
    z_tot_mat = (f_H2p[:, np.newaxis] * bitstrings[:, [2*p]] * bitstrings)

    # Compute the sums for y_tot and z_tot across a_idx (axis 0)
    y_tot_vec = np.sum(y_tot_mat, axis=0)
    z_tot_vec = np.sum(z_tot_mat, axis=0)

    # Compute nu_y and nu_z using Gamma_vec
    nu_y = -np.sum(Gamma_vec * y_tot_vec**2)
    nu_z = np.sum(Gamma_vec * z_tot_vec**2)

    if separate: return np.real(1j*nu_y/2), np.real(1j*nu_z/2)
    else: return np.real(1j*nu_y/2 + 1j*nu_z/2)
    

def get_xy_nu_helper(p, separate=False, negate=False, force_normal=False):
    bitstrings = gen_all_bitstrings(4*p+1)

    def get_expectation(params):
        gamma_y= params[:p]
        gamma_z = params[p:2*p]
        beta = params[2*p:]
        beta = np.concatenate((beta, np.zeros(1)))
        if force_normal:
            gamma_y = np.abs(gamma_y)
            gamma_z = -np.abs(gamma_z)
            beta = np.abs(beta)
        return (1-2*negate) * get_xy_nu(p, gamma_y, gamma_z, beta, bitstrings, separate=separate)

    return get_expectation

#### Parameter Optimization

##### $MC$ Ansatz

In [ ]:
mc_ps = [1,2,3,4,5,6,7]
mc_best_xs = []
mc_best_nus = []

coeffs= [-1,-1]

for p in mc_ps:
    trials=p**2
    print(f'p={p}')
    cost_fn = get_mc_nu_helper(p, negate=True) 
    best_nu, best_x = 0, np.zeros(2*p)
    for trial in tqdm(range(trials)):
        init = (np.random.random(2*p))/2-(np.random.random(2*p))/2
        res = minimize(cost_fn, init, method='bfgs', options={'gtol':1e-4})
        if -res.fun > best_nu:
            best_nu = -res.fun
            print(f' v_p={np.round(best_nu,6)}, x={np.round(res.x,4)}')
            best_x = res.x
    mc_best_xs.append(best_x)
    mc_best_nus.append(best_nu)
    print(f'FINAL p={p}, v_p={np.round(best_nu,6)}, x={np.round(best_x,4)}')
    print()

In [ ]:
mc_large_ps = [8,9,10]
mc_large_best_xs = []
mc_large_best_nus = []

coeffs= [-1,-1]

for p in mc_large_ps:
    cost_fn = get_mc_nu_helper(p, negate=True) 
    init = np.concatenate((stretch_array(mc_best_xs[-1][:p-1], p), stretch_array(mc_best_xs[-1][p-1:], p)))
    res = minimize(cost_fn, init, method='bfgs', options={'gtol':1e-4})
    mc_best_xs.append(res.x)
    mc_best_nus.append(-res.fun)
    print(f'FINAL p={p}, v_p={np.round(-res.fun,6)}, x={np.round(res.x,4)}')
    print()

In [ ]:
with open('data/mc_best_nus.pickle', 'wb') as handle:
    pickle.dump(mc_best_nus, handle)
with open('data/mc_best_xs.pickle', 'wb') as handle:
    pickle.dump(mc_best_xs, handle)

In [ ]:
with open('data/mc_best_nus.pickle', 'rb') as handle:
    mc_best_nus = pickle.load(handle)
with open('data/mc_best_xs.pickle', 'rb') as handle:
    mc_best_xs = pickle.load(handle)

print(mc_best_nus)
print(mc_best_xs)

In [ ]:
best_nus_last_gamma_only = []
for p in [2,3,4,5,6,7,8,9,10]:
    cost_fn = get_mc_nu_last_gamma_helper(p, sk_angles[p-1][0], sk_angles[p-1][1], separate=False, negate=True)
    res = minimize(cost_fn, np.random.random(1))
    print(p, res.x, -res.fun)
    best_nus_last_gamma_only.append(-res.fun)

In [ ]:
mc_energies_farhi = [.3033, .4075, .4726, .5157, .5476, .5721, .5915, 0.6073, 0.6203, 0.6314]

In [ ]:
for p in [2,3,4,5,6,7,8,9,10]:
    print(f'p={p}: full optimized nu={np.round(mc_best_nus[p-1],4)}, last gamma optimized nu={np.round(best_nus_last_gamma_only[p-2],4)}, mc best nu at (p-1)={mc_energies_farhi[p-2]}', )

In [ ]:
with open('data/mc_last_gamma_best_nus.pickle', 'wb') as handle:
    pickle.dump(best_nus_last_gamma_only, handle)

##### $XY$ ansatz

In [ ]:
xy_ps = [2,3,4]
xy_best_xs = []
xy_best_nus = []
coeffs= [-1,-1]
trials = 50
for p in xy_ps:
    print(f'p={p}')
    cost_fn = get_xy_nu_helper(p, negate=True) 
    best_nu, best_x = 0, np.zeros(3*p)
    for trial in tqdm(range(trials)):
        init = (np.random.random(3*p))-(np.random.random(3*p))
        res = minimize(cost_fn, init, method='bfgs', options={'gtol':1e-4})
        if -res.fun > best_nu:
            best_nu = -res.fun
            print(f' v_p={np.round(best_nu,6)}, x={np.round(res.x,4)}')
            best_x = res.x
    xy_best_xs.append(best_x)
    xy_best_nus.append(best_nu)
    print(f'FINAL p={p}, v_p={np.round(best_nu,6)}, x={np.round(best_x,4)}')
    print()

In [ ]:
xy_ps = [4]
xy_best_xs = []
xy_best_nus = []
coeffs= [-1,-1]

for p in xy_ps:
    trials = 10
    print(f'p={p}')
    cost_fn = get_xy_nu_helper(p, negate=True) 
    best_nu, best_x = 0, np.zeros(3*p)
    for trial in tqdm(range(trials)):
        init = (np.random.random(3*p))/2-(np.random.random(3*p))/2
        res = minimize(cost_fn, init, method='bfgs', options={'gtol':1e-4})
        if -res.fun > best_nu:
            best_nu = -res.fun
            print(f' v_p={np.round(best_nu,6)}, x={np.round(res.x,4)}')
            best_x = res.x
    xy_best_xs.append(best_x)
    xy_best_nus.append(best_nu)
    print(f'FINAL p={p}, v_p={np.round(best_nu,6)}, x={np.round(best_x,4)}')
    print()

In [ ]:
with open('data/xy_best_nus.pickle', 'wb') as handle:
    pickle.dump(xy_best_nus, handle)
with open('data/xy_best_xs.pickle', 'wb') as handle:
    pickle.dump(xy_best_xs, handle)

In [ ]:
with open('data/xy_best_nus.pickle', 'rb') as handle:
    xy_best_nus = pickle.load(handle)
with open('data/xy_best_xs.pickle', 'rb') as handle:
    xy_best_xs = pickle.load(handle)

xy_best_nus, xy_best_xs

#### Testing agreement 

In [ ]:
# testing YZ with 0 gamma_y equivalence with MC
import numpy as np
def test_mc_and_xy_equal(p, d=np.inf, rtol=.01):

    mc_cost_fn = get_mc_nu_helper(p, separate=True)
    xy_cost_fn = get_xy_nu_helper(p, separate=True)
    #random angles
    angles = np.random.random(2*p)
    random_mc = mc_cost_fn(angles)
    random_xy = xy_cost_fn(np.concatenate((np.zeros(p), angles)))
    random_rdif = np.round(np.sum(np.abs(np.array([random_mc])-np.array([random_xy])))/np.sum(np.abs(random_mc)),4)
    if np.allclose(random_mc, random_xy, rtol=rtol): 
        print(f'passed random: mc={np.round(random_mc,6)}, xy={np.round(random_xy,6)}, rdif={random_rdif}, angles={angles}')
    else:
        print(f'failed random: mc={np.round(random_mc,6)}, xy={np.round(random_xy,6)}, rdif={random_rdif}')
    return

for p in [1,2,3,4]:
    print(f'p={p}')
    test_mc_and_xy_equal(p, d=np.inf)

### Finite $D$

##### $MC$ ansatz implementation

In [ ]:
def get_mc_gamma_vec(gamma):
    return  np.concatenate((gamma, np.array([0]), -gamma[::-1]))

def get_mc_gamma_dp_matrix(gamma, bitstrings):
    gamma_vec = get_mc_gamma_vec(gamma) 
    # Compute the element-wise product and sum (dot products) for all pairs using matrix multiplication
    bs_product_matrix = np.dot(bitstrings * gamma_vec, bitstrings.T)
    return bs_product_matrix

def get_mc_Hps(p, gamma_dp_matrix, fas, d):
    bs_size = 2 * p + 1
    H_prev = np.ones(2**bs_size, dtype=complex)
    for _ in range(p):
        cos_matrix = np.cos((1 / np.sqrt(d)) * gamma_dp_matrix)
        total_matrix = fas * H_prev * cos_matrix
        H_new = np.sum(total_matrix, axis=1) **d
        H_prev = H_new 
    return H_new


def get_mc_expectation_helper(p, separate=False, d=None, coeffs=np.ones(3)):
    bitstrings = gen_all_bitstrings(2*p+1)

    def get_expectation(params):
        gamma = params[:p]
        beta = params[p:]
        assert len(gamma) == p
        assert len(beta) == p

        fas = get_mc_f_beta(p, bitstrings, beta)
        fpas = get_mc_f_beta(p, bitstrings, beta, get_f_prime=True)
        
        gamma_dp_matrix = get_mc_gamma_dp_matrix(gamma, bitstrings)
        gamma_inside_matrix = (1 / np.sqrt(d)) * gamma_dp_matrix
        Hp = get_mc_Hps(p, gamma_dp_matrix,  fas, d)
        
        bs1_p = bitstrings[:, p]
        bs2_p = bitstrings[:, p].reshape(-1, 1)
        
        fas_outer = fas[:, np.newaxis] * fas
        fpas_outer = fpas[:, np.newaxis] * fpas
        Hp_outer = Hp[:, np.newaxis] * Hp
        
        sin_gamma_inside_matrix = np.sin(gamma_inside_matrix)
        cos_gamma_inside_matrix = np.cos(gamma_inside_matrix)
        
        totalXX = coeffs[0] * np.sum(fpas_outer * cos_gamma_inside_matrix * Hp_outer)
        totalYY = coeffs[1] * np.sum((-1j) * bs1_p * bs2_p * fpas_outer * sin_gamma_inside_matrix * Hp_outer)
        totalZZ = coeffs[2] * np.sum(1j * bs1_p * bs2_p * fas_outer * sin_gamma_inside_matrix * Hp_outer)
        
        if separate: 
            return np.array([np.real(totalXX), np.real(totalYY), np.real(totalZZ)])
        
        else:
            return  np.real(totalXX + totalYY + totalZZ)

    return get_expectation

##### Optimization

see scaling with p, convergence to something?

In [ ]:
ps=[1,2,3,4,5]
ds=[1,2]
best_Es_qmc, best_Es_epr, best_Es_xy = np.zeros((len(ds), len(ps))), np.zeros((len(ds), len(ps))), np.zeros((len(ds), len(ps)))
all_best_xs_qmc, all_best_xs_epr, all_best_xs_xy = [], [], []
for d_idx, d in enumerate(ds):
    print(f'd={d}')
    d_best_xs_qmc, d_best_xs_epr, d_best_xs_xy = [], [], []
    for p_idx, p in enumerate(ps):
        print(f'p={p}')
        qmc_cost_fn = get_mc_expectation_helper(p, separate=False, d=d, coeffs=np.array([1,1,1])) # minimize +XX+YY+ZZ = maximize -XX-YY-ZZ
        epr_cost_fn = get_mc_expectation_helper(p, separate=False, d=d, coeffs=np.array([-1,1,-1])) # minimize -XX+YY-ZZ = maximize +XX-YY+ZZ
        xy_cost_fn = get_mc_expectation_helper(p, separate=False, d=d, coeffs=np.array([0,1,1])) # minimize +YY+ZZ = maximize -YY-ZZ
        best_E_qmc, best_xs_qmc = 0, np.zeros(2*p) 
        best_E_epr, best_xs_epr = 0, np.zeros(2*p) 
        best_E_xy, best_xs_xy = 0, np.zeros(2*p) 
        for _ in range(20):
            init = np.random.random(2*p)-np.random.random(2*p)
            res_qmc = minimize(qmc_cost_fn, init)
            res_epr = minimize(epr_cost_fn, init)
            res_xy = minimize(xy_cost_fn, init)
            E_qmc = 1/2-res_qmc.fun/2
            E_epr = 1/2-res_epr.fun/2
            E_xy = 1/2-res_xy.fun/2
            if E_qmc > best_E_qmc:
                best_E_qmc = E_qmc
                best_xs_qmc = res_qmc.x
            if E_epr > best_E_epr:
                best_E_epr = E_epr
                best_xs_epr = res_epr.x
            if E_xy > best_E_xy:
                best_E_xy = E_xy
                best_xs_xy = res_xy.x
        print(f'best_E qmc: {best_E_qmc}, best E epr: {best_E_epr}, best E xy: {best_E_xy}')
        best_Es_qmc[d_idx, p_idx] = best_E_qmc
        best_Es_epr[d_idx, p_idx] = best_E_epr
        best_Es_xy[d_idx, p_idx] = best_E_xy
        d_best_xs_qmc.append(best_xs_qmc)
        d_best_xs_epr.append(best_xs_epr)
        d_best_xs_xy.append(best_xs_xy)
    all_best_xs_qmc.append(d_best_xs_qmc)
    all_best_xs_epr.append(d_best_xs_epr)
    all_best_xs_xy.append(d_best_xs_xy)

In [ ]:
print(all_best_xs_qmc)
print(all_best_xs_epr)
print(all_best_xs_xy)

In [ ]:
ps=[1,2,3,4,5]
ds=[3,4]
best_Es_qmc, best_Es_epr, best_Es_xy = np.zeros((len(ds), len(ps))), np.zeros((len(ds), len(ps))), np.zeros((len(ds), len(ps)))
all_best_xs_qmc, all_best_xs_epr, all_best_xs_xy = [], [], []
for d_idx, d in enumerate(ds):
    print(f'd={d}')
    d_best_xs_qmc, d_best_xs_epr, d_best_xs_xy = [], [], []
    for p_idx, p in enumerate(ps):
        print(f'p={p}')
        qmc_cost_fn = get_mc_expectation_helper(p, separate=False, d=d, coeffs=np.array([1,1,1])) # minimize +XX+YY+ZZ = maximize -XX-YY-ZZ
        epr_cost_fn = get_mc_expectation_helper(p, separate=False, d=d, coeffs=np.array([-1,1,-1])) # minimize -XX+YY-ZZ = maximize +XX-YY+ZZ
        xy_cost_fn = get_mc_expectation_helper(p, separate=False, d=d, coeffs=np.array([0,1,1])) # minimize +YY+ZZ = maximize -YY-ZZ
        best_E_qmc, best_xs_qmc = 0, np.zeros(2*p) 
        best_E_epr, best_xs_epr = 0, np.zeros(2*p) 
        best_E_xy, best_xs_xy = 0, np.zeros(2*p) 
        for _ in range(20):
            init = np.random.random(2*p)-np.random.random(2*p)
            res_qmc = minimize(qmc_cost_fn, init)
            res_epr = minimize(epr_cost_fn, init)
            res_xy = minimize(xy_cost_fn, init)
            E_qmc = 1/2-res_qmc.fun/2
            E_epr = 1/2-res_epr.fun/2
            E_xy = 1/2-res_xy.fun/2
            if E_qmc > best_E_qmc:
                best_E_qmc = E_qmc
                best_xs_qmc = res_qmc.x
            if E_epr > best_E_epr:
                best_E_epr = E_epr
                best_xs_epr = res_epr.x
            if E_xy > best_E_xy:
                best_E_xy = E_xy
                best_xs_xy = res_xy.x
        print(f'best_E qmc: {best_E_qmc}, best E epr: {best_E_epr}, best E xy: {best_E_xy}')
        best_Es_qmc[d_idx, p_idx] = best_E_qmc
        best_Es_epr[d_idx, p_idx] = best_E_epr
        best_Es_xy[d_idx, p_idx] = best_E_xy
        d_best_xs_qmc.append(best_xs_qmc)
        d_best_xs_epr.append(best_xs_epr)
        d_best_xs_xy.append(best_xs_xy)
    all_best_xs_qmc.append(d_best_xs_qmc)
    all_best_xs_epr.append(d_best_xs_epr)
    all_best_xs_xy.append(d_best_xs_xy)

In [ ]:
print(all_best_xs_qmc)
print(all_best_xs_epr)
print(all_best_xs_xy)

In [ ]:
ps = np.arange(1,6)
# Given data

y_values = 2*np.log(2) - np.array([.5, 0.8119196015383394 , 0.9117302730070458, 1.0133793462433751,  1.055389329879501])

# Fit a linear polynomial to the data (degree 1)
coefficients, cov_matrix = np.polyfit(1/ps, y_values, 1, cov=True)
polynomial_fit = np.polyval(coefficients, 1/ps)

# Extract slope, intercept and their uncertainties (square root of diagonal elements of covariance matrix)
slope, intercept = coefficients
slope_err, intercept_err = np.sqrt(np.diag(cov_matrix))

# Compute R-squared
y_pred = np.polyval(coefficients, 1/ps)
ss_res = np.sum((y_values - y_pred) ** 2)
ss_tot = np.sum((y_values - np.mean(y_values)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

# Create the plot
fig, ax = plt.subplots(nrows=1, ncols=1)

# Scatter plot with data points
ax.plot(1/ps, y_values, 'o--', label=fr'$MC$ ansatz on $H_{{XY}}$', markersize=5, linewidth=.5)

# Add the polynomial fit
fit_label = fr'Poly fit'
ax.plot(1/ps, polynomial_fit, '--', label=fit_label, color='grey')

# Display R^2 value
ax.text(0.5, 0.8, f'$R^2 = {r_squared:.3f}$', transform=ax.transAxes, fontsize=12)

# Set axis labels
ax.set_ylabel(r'$2log2-E$', fontsize=13, rotation=90, labelpad=12)
ax.set_xlabel(r'$\frac{1}{p}$', fontsize=14)

# Legend and grid
ax.legend(loc='lower right', fontsize=11, framealpha=1)
#ax.set_xscale('log')
#ax.set_yscale('log')
ax.grid()

# Adjust layout and save the figure
plt.tight_layout()
#plt.subplots_adjust(bottom=0.15)


plt.show()

print(slope, slope_err, intercept, intercept_err, r_squared)
print(fr'$2log2-E$ = ({slope:.3e} $\pm$ {slope_err:.3e})(1/p) + ({intercept:.3e} $\pm$ {intercept_err:.3e}')

##### $XY$ ansatz implementation

In [ ]:
def get_xy_H2p_finite_D(p, gamma_dp_matrix, fas, d):
    bs_size = 4 * p + 1
    H_prev = np.ones(2**bs_size, dtype=complex)
    for _ in range(2*p):
        cos_matrix = np.cos((1 / np.sqrt(d)) * gamma_dp_matrix)
        total_matrix = fas * H_prev * cos_matrix
        H_new = np.sum(total_matrix, axis=1) **d
        H_prev = H_new 
    return H_new


def get_xy_gamma_dp_matrix(gamma_vec, bitstrings):
    return np.dot(bitstrings * gamma_vec, bitstrings.T)

def get_xy_expectation_helper(p, separate=False, d=None, coeffs=(1,1,1)):

    bitstrings = gen_all_bitstrings(4*p+1)
    def get_expectation(params):
        gamma_y= params[:p]
        gamma_z = params[p:2*p]
        beta = params[2*p:]
        gamma_vec = np.zeros(4*p+1)
        gamma_vec[:2*p:2]=gamma_z
        gamma_vec[1:2*p:2]=gamma_y
        gamma_vec[4*p-1:2*p:-2]=-gamma_y
        gamma_vec[4*p:2*p:-2]=-gamma_z

        fas = get_xy_f(p, bitstrings, beta)
        fpas = get_xy_f(p, bitstrings, beta, f_prime=True)

        gamma_dp_matrix = get_xy_gamma_dp_matrix(gamma_vec, bitstrings)
        gamma_inside_matrix = (1 / np.sqrt(d)) * gamma_dp_matrix
        H2p = get_xy_H2p_finite_D(p, gamma_dp_matrix, fas, d) # get H^(2p)

        bs1_2p = bitstrings[:, 2*p]
        bs2_2p = bitstrings[:, 2*p].reshape(-1, 1)
        
        fas_outer = fas[:, np.newaxis] * fas
        fpas_outer = fpas[:, np.newaxis] * fpas
        H2p_outer = H2p[:, np.newaxis] * H2p

        sin_gamma_inside_matrix = np.sin(gamma_inside_matrix)
        cos_gamma_inside_matrix = np.cos(gamma_inside_matrix)
        
        totalXX = coeffs[0] * np.sum(fpas_outer * cos_gamma_inside_matrix * H2p_outer)
        totalYY = coeffs[1] * np.sum((-1j) * bs1_2p * bs2_2p * fpas_outer * sin_gamma_inside_matrix * H2p_outer)
        totalZZ = coeffs[2] * np.sum(1j * bs1_2p * bs2_2p * fas_outer * sin_gamma_inside_matrix * H2p_outer)
        
        if separate: 
            return np.array([np.real(totalXX), np.real(totalYY), np.real(totalZZ)])
        
        else:
            return  np.real(totalXX + totalYY + totalZZ)

    return get_expectation

##### Optimization

In [ ]:
# testing agreement
p=2
d=2
qmc_cost_fn_mc = get_mc_expectation_helper(p, separate=True, d=d, coeffs=np.array([1,1,1])) # minimize +XX+YY+ZZ = maximize -XX-YY-ZZ
qmc_cost_fn_xy = get_xy_expectation_helper(p, separate=True, d=d, coeffs=np.array([1,1,1])) # minimize +XX+YY+ZZ = maximize -XX-YY-ZZ
print(qmc_cost_fn_mc(np.array([1,.5,1,1])))
print(qmc_cost_fn_xy(np.array([0,0,1,.5,1,1]))) # all gamma_y = 0 

In [ ]:
ps=[1,2]
ds=[1,2,3,4]
best_Es_qmc, best_Es_epr, best_Es_xy = np.zeros((len(ds), len(ps))), np.zeros((len(ds), len(ps))), np.zeros((len(ds), len(ps)))
all_best_xs_qmc, all_best_xs_epr, all_best_xs_xy = [], [], []
for d_idx, d in enumerate(ds):
    print(f'd={d}')
    d_best_xs_qmc, d_best_xs_epr, d_best_xs_xy = [], [], []
    for p_idx, p in enumerate(ps):
        print(f'p={p}')
        qmc_cost_fn = get_xy_expectation_helper(p, separate=False, d=d, coeffs=np.array([1,1,1])) # minimize +XX+YY+ZZ = maximize -XX-YY-ZZ
        epr_cost_fn = get_xy_expectation_helper(p, separate=False, d=d, coeffs=np.array([-1,1,-1])) # minimize -XX+YY-ZZ = maximize +XX-YY+ZZ
        xy_cost_fn = get_xy_expectation_helper(p, separate=False, d=d, coeffs=np.array([0,1,1])) # minimize +YY+ZZ = maximize -YY-ZZ
        best_E_qmc, best_xs_qmc = 0, np.zeros(3*p) 
        best_E_epr, best_xs_epr = 0, np.zeros(3*p) 
        best_E_xy, best_xs_xy = 0, np.zeros(3*p) 
        for _ in tqdm(range(100//p**2)): # takes very long to run p=2
            init = np.random.random(3*p)-np.random.random(3*p)
            res_qmc = minimize(qmc_cost_fn, init)
            res_epr = minimize(epr_cost_fn, init)
            res_xy = minimize(xy_cost_fn, init)
            E_qmc = 1/2-res_qmc.fun/2
            E_epr = 1/2-res_epr.fun/2
            E_xy = 1/2-res_xy.fun/2
            if E_qmc > best_E_qmc:
                best_E_qmc = E_qmc
                best_xs_qmc = res_qmc.x
            if E_epr > best_E_epr:
                best_E_epr = E_epr
                best_xs_epr = res_epr.x
            if E_xy > best_E_xy:
                best_E_xy = E_xy
                best_xs_xy = res_xy.x
        print(f'best E qmc: {best_E_qmc}, best E epr: {best_E_epr}, best E xy: {best_E_xy}')
        print(f'best xs qmc: {np.round(best_xs_qmc,6)}, best xs epr: {np.round(best_xs_epr,6)}, best xs xy: {np.round(best_xs_xy, 6)}')
        best_Es_qmc[d_idx, p_idx] = best_E_qmc
        best_Es_epr[d_idx, p_idx] = best_E_epr
        best_Es_xy[d_idx, p_idx] = best_E_xy
        d_best_xs_qmc.append(best_xs_qmc)
        d_best_xs_epr.append(best_xs_epr)
        d_best_xs_xy.append(best_xs_xy)
    all_best_xs_qmc.append(d_best_xs_qmc)
    all_best_xs_epr.append(d_best_xs_epr)
    all_best_xs_xy.append(d_best_xs_xy)